## UsersKFoldLeavePOut validation

https://drive.google.com/uc?id=1GIa7RQOTXqvvwFFckfTVVNrtTYvlQwzj## Homework

Исходные данные - Yandex Cup 2022 RecSys:
- Описание соревнования - https://contest.yandex.ru/yacup/contest/41618/problems/
- Данные - https://disk.yandex.ru/d/SI1aAooPn9i8TA
- Описание данных - в архиве likes_data.zip три файла:
  - train - обучающий набор данных. Каждая строка - последовательность id треков, которые лайкнул один пользователь. Гарантируется, что лайки даны в той последовательности, в которой их ставил пользователь.
  - test - набор тестовых данных. Имеет точно такой же формат, но в каждой строке не хватает последнего лайка, который надо предсказать.
  - track_artists.csv - информация о исполнителях треков. Гарантируется, что у каждого трека есть ровно один исполнитель. Для треков, у которых фактически несколько исполнителей, мы оставили того, который считается основным исполнителем трека.
- Описание сабмита - в качестве решения необходимо отправить файл, в котором для каждого пользователя из test в отдельной строке будет не более 100 треков, разделенных пробелом. Гарантируется, что у каждого пользователя будет только 1 лайк в тесте
- Метрика - MRR@100

In [3]:
import numpy as np
import pandas as pd

In [13]:
LIMIT = 1_000_00

In [14]:
data = []

with open('train', 'r') as f:
    for i, line in zip(range(LIMIT), f):
        data.extend([dict(user_id=i, item_id=int(item_id))
                     for item_id in reversed(line.split())])

In [15]:
data = pd.DataFrame(data, columns=['user_id', 'item_id'])
data.head()

,user_id,item_id
0,0,333396
1,0,267089
2,0,155959
3,0,353335
4,0,414000


In [16]:
class UsersKFoldPOut():
    """
    Users `K` fold `P` out validation schema.
    
    Attributes:
        - n_folds: count of folds.
        - p: leave at least P interactions in the test for each user
        - random_seed: random seed.
        - user_column: user's column.
    """
    
    def __init__(self,
                 n_folds: int,
                 p: int,
                 random_seed=23,
                 user_column='user_id'
                 ):
        
        self.n_folds = n_folds
        self.random_seed = random_seed
        self.user_column = user_column 
        self.p = p
    
    def split(self, df):
        users = df[self.user_column].unique()
        users_count = len(users)
        
        np.random.seed(self.random_seed)
        np.random.shuffle(users)
        
        fold_sizes = np.full(
            self.n_folds,
            users_count // self.n_folds,
            dtype=int
        )
        
        fold_sizes[:users_count % self.n_folds] += 1
        
        current = 0
        like_order = df.groupby(self.user_column).cumcount()
        
        for fold_size in fold_sizes:
            start, stop = current, current + fold_size
            test_fold_users = users[start:stop]
            test_mask = df[self.user_column].isin(test_fold_users) & (like_order < self.p)
            train_mask = ~df[self.user_column].isin(test_fold_users)
            
            yield train_mask, test_mask

In [17]:
p = 5
cv = UsersKFoldPOut(n_folds=3, p=p)

for i, (train_mask, test_mask) in enumerate(cv.split(data)):
    train = data[train_mask]
    test = data[test_mask]
    
    print(f'Fold#{i} | Train: {train.shape[0]}, Test: {test.shape[0]}')

    assert len(set(train['user_id'].unique()).intersection(test['user_id'].unique())) == 0, 'Intersection of train&test detected.'
    assert test.groupby('user_id').count().max()[0] <= p, 'Test part has more than p items'

Fold#0 | Train: 5427560, Test: 166670
Fold#1 | Train: 5427571, Test: 166665
Fold#2 | Train: 5427571, Test: 166665
